In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
languages_df = spark.read.parquet(f"{silver_folder_path}/languages")

In [0]:
movies_languages_df = spark.read.parquet(f"{silver_folder_path}/movies_languages")

In [0]:
genres_df = spark.read.parquet(f"{silver_folder_path}/genre")

In [0]:
movies_genres_df = spark.read.parquet(f"{silver_folder_path}/movie_genre")

#### Join "languages" y "movies lenguages"

In [0]:
language_mov_lan_df = languages_df.join(movies_languages_df, on='language_id', how='inner') \
    .select(languages_df.language_name, movies_languages_df.movie_id)


#### Join "genre" y "movie_genre"

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df, on='genre_id', how='inner') \
                    .select(genres_df.genre_name, movies_genres_df.movie_id)

#### Join "movies_df", "language_mov_lan_df", y "genres_mov_gen_df"

* Filtrar las peliculas donde su fecha de lanzamiento sea mayor o igual a 2000

In [0]:
movie_filter_df = movies_df.filter(movies_df.year_release_date >= 2000)

In [0]:
results_movie_genre_languages_df = movie_filter_df \
    .join(language_mov_lan_df, on='movie_id', how='inner') \
    .join(genres_mov_gen_df, on='movie_id', how='inner') 

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
results_df = results_movie_genre_languages_df \
    .select("title", "duration_time", "release_date", "vote_average", "language_name", "genre_name") \
    .withColumn('create_date', current_timestamp())

ordenar por la columna "release_date" de manera descendente

In [0]:
results_df.orderBy("release_date", ascending=False)

In [0]:
results_df.display()

### Escribir en el data lake en formato parquet

In [0]:
results_df.write.parquet(f"{gold_folder_path}/results_movie_genre_languages", mode="overwrite")

In [0]:
spark.read.parquet(f"{gold_folder_path}/results_movie_genre_languages").display()